In [1]:
import sys
sys.path.append("..")

Fine tune a pretrained DistilBert model on dataset (`conll2003`) for named entity recognition. This will be our basic starting model.

In [4]:
from src.models.train_model import train_model
from src.features.build_features import load_token_class_dataset

dataset = load_token_class_dataset()
model_trainer = train_model()

/Users/michellelo/Desktop/research/neuron-identification-1/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset conll2003 (/Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 56.47it/s]
Found cached dataset conll2003 (/Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 478.60it/s]
Found cached dataset conll2003 (/Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 206.85it/s]
Loading cached processed dataset at /Users/michellel

In [ ]:
from src.models.evaluate_model import evaluate
from src.visualization.visualize import get_basic_model

# Train and evaluate a basic model on pretrained DistilBert
basic_model = get_basic_model(model_trainer)
evaluate(basic_model, dataset["validation"])

Identify concept neurons in the basic model associated with location names.

In [2]:
from src import basic_model_path, basic_activations_path
from src.visualization.analyse_model import analyse_model

# Identify neurons in the basic model to ablate
basic_analyser = analyse_model(basic_model_path, basic_activations_path)

/Users/michellelo/Desktop/research/neuron-identification-1/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model: ../models/basic_model,distilbert-base-cased


Some weights of the model checkpoint at ../models/basic_model were not used when initializing DistilBertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Reading input corpus
Preparing output file
Extracting representations from model
Sentence         : "SITTING more than 700m above sea level , Mt Diogenes , as it is formally known , was given the Hanging Rock moniker thanks to a precariously suspended boulder , perched in the volcanic mass of the rocky Victorian fortress ."
Original    (041): ['SITTING', 'more', 'than', '700m', 'above', 'sea', 'level', ',', 'Mt', 'Diogenes', ',', 'as', 'it', 'is', 'formally', 'known', ',', 'was', 'given', 'the', 'Hanging', 'Rock', 'moniker', 'thanks', 'to', 'a', 'precariously', 'suspended', 'boulder', ',', 'perched', 'in', 'the', 'volcanic', 'mass', 'of', 'the', 'rocky', 'Victorian', 'fortress', '.']
Tokenized   (052): ['[CLS]', 'S', '##IT', '##TI', '##NG', 'more', 'than', '700', '##m', 'above', 'sea', 'level', ',', 'Mt', 'Di', '##ogen', '##es', ',', 'as', 'it', 'is', 'formally', 'known', ',', 'was', 'given', 'the', 'Hang', '##ing', 'Rock', 'moniker', 'thanks', 'to', 'a', 'pre', '##car', '##iously', 's

Sentence         : "Samantha Stevenson of Toronto was charged with pretending to practice witchcraft , fraud over $ 5,000 and possession of property obtained by a crime , according to the York Regional Police ."
Original    (032): ['Samantha', 'Stevenson', 'of', 'Toronto', 'was', 'charged', 'with', 'pretending', 'to', 'practice', 'witchcraft', ',', 'fraud', 'over', '$', '5,000', 'and', 'possession', 'of', 'property', 'obtained', 'by', 'a', 'crime', ',', 'according', 'to', 'the', 'York', 'Regional', 'Police', '.']
Tokenized   (037): ['[CLS]', 'Samantha', 'Stevenson', 'of', 'Toronto', 'was', 'charged', 'with', 'pretending', 'to', 'practice', 'witch', '##craft', ',', 'fraud', 'over', '$', '5', ',', '000', 'and', 'possession', 'of', 'property', 'obtained', 'by', 'a', 'crime', ',', 'according', 'to', 'the', 'York', 'Regional', 'Police', '.', '[SEP]']
Filtered   (035): ['Samantha', 'Stevenson', 'of', 'Toronto', 'was', 'charged', 'with', 'pretending', 'to', 'practice', 'witch', '##craft', ','

In [ ]:
neurons_to_prune = basic_analyser.identify_concept_neurons()

In [ ]:
basic_analyser.show_top_words(neurons_to_prune)

Prune the identified concept neurons from the model by setting their weights to zero.

In [ ]:
from src import basic_model_path, pruned_model_path
from src.models.prune_model import prune_model
from src.models.evaluate_model import evaluate
from src.visualization.visualize import get_pruned_model

# If the model has been saved
pruned_model = get_pruned_model(pruned_model_path, model_trainer)

# Ablate neurons
# pruned_model = prune_model(basic_model_path, model_trainer, neurons_to_prune)
# pruned_model.save_pretrained(pruned_model_path)
evaluate(pruned_model, dataset["validation"])

Retrain the pruned model until performance recovers. Examine:

- In which neurons does the concept of location names reappear in the pruned model?
- Is there any relation between the new concept (location names) and what concepts originally existed in these neurons?

In [ ]:
from src import pruned_model_path, retrained_model_path
from src.visualization.visualize import get_retrained_model

# Retrain the pruned model
retrained_model = get_retrained_model(retrained_model_path, pruned_model_path, model_trainer)

In [ ]:
from src import retrained_model_path, retrained_activations_path
from src.visualization.analyse_model import analyse_model

# Examine the retrained model for concepts and compare with old model
retrained_analyser = analyse_model(retrained_model_path, retrained_activations_path)

In [ ]:
new_concept_neurons = retrained_analyser.identify_concept_neurons()

In [ ]:
retrained_analyser.show_top_words(new_concept_neurons)
basic_analyser.show_top_words(new_concept_neurons)